In [6]:
import numpy as np
import pickle 
import pandas as pd
WORK_PATH = "/Users/welcome/Google Drive (cuong.tranus@gmail.com)/research/internship/fraud-detection"
feat_dict = pickle.load(open(WORK_PATH + "/results/features_category.pkl" ,"rb"))

numeric_cols = feat_dict['numeric_cols']
cat_cols = feat_dict['cat_cols']

print(cat_cols)

['creditLimit', 'posEntryMode', 'posConditionCode', 'merchantCategoryCode', 'transactionType', 'cardPresent', 'isFraud']


In [3]:
pd00 = pd.read_csv(WORK_PATH + "/data/processed_data.csv")
print(pd00.shape)

(786363, 26)


In [4]:
pd00.head(2)

accountNumber  customerId  creditLimit  availableMoney  \
0      737265056   737265056         5000          5000.0   
1      737265056   737265056         5000          5000.0   

   transactionDateTime  transactionAmount merchantName  posEntryMode  \
0  2016-08-13T14:27:32              98.55         Uber           2.0   
1  2016-10-11T05:05:54              74.51  AMC #191138           9.0   

   posConditionCode merchantCategoryCode  ... transactionType currentBalance  \
0               1.0            rideshare  ...        PURCHASE            0.0   
1               1.0        entertainment  ...        PURCHASE            0.0   

  cardPresent  isFraud        date  tvt_code_0 tvt_code_1  tvt_code_2  \
0       False        0  2016-08-13     train_0    train_1     train_2   
1        True        0  2016-10-11        test       test        test   

   tvt_code_3  tvt_code_4  
0     train_3       val_4  
1        test        test  

[2 rows x 26 columns]

In [16]:
for col in cat_cols:
    if col!='isFraud':
        print( pd00.groupby(col).agg({'isFraud':[np.mean, np.size]}).reset_index())
        print("----------------------------------------------------------------")

  creditLimit   isFraud        
                   mean    size
0         250  0.011962   34025
1         500  0.017161   27097
2        1000  0.014988   36430
3        2500  0.014795   75429
4        5000  0.017472  201863
5        7500  0.013175   97913
6       10000  0.013465   56889
7       15000  0.018269  139307
8       20000  0.013318   68629
9       50000  0.017240   48781
----------------------------------------------------------------
  posEntryMode   isFraud        
                    mean    size
0          2.0  0.017460  195934
1          5.0  0.007796  315035
2          9.0  0.023930  236481
3         80.0  0.015704   15283
4         90.0  0.019003   19576
----------------------------------------------------------------
  posConditionCode   isFraud        
                        mean    size
0              1.0  0.016269  628787
1              8.0  0.012938  149634
2             99.0  0.030400    7533
----------------------------------------------------------------
    m

In [17]:
################################################################################
###
################################################################################
import os, re, sys, collections, datetime, dateutil
import numpy as np, pandas as pd
import importlib
import math
import calendar


################################################################################
### General functions
################################################################################
def describe_spark_df(df):
    """
    Summarize pyspark dataframe. df should be cached already
    """
    print("### Table schema & Sample. Count: %d ###" % (df.count()))
    x1 = df.take(1)[0]
    x1d = x1.asDict()
    for w in sorted(df.dtypes, key=lambda x: x[0]): print(w[0], w[1], x1d[w[0]])


#
def describe_dict(dictvar, title):
    """
    Summarize a python dictionary
    """
    print("### %s ###" % str(title))
    keylist = sorted(dictvar.keys())
    for k in keylist: print("  " + str((k, dictvar[k])))
    return


#
def split_bool_arr(arr_ind, arr_ratio_one):
    """
    Args:
        arr_ind: binary array. Assume that non-zero elements are choosable boolean indices
    Split arr_ind into two binary arrays: res_ind01 and res_ind00
    Returns:
        res_ind01, res_ind00: binary arrays of similar shape of arr_ind, satisfying
            arr_ind[i]=0 => res_ind01[i]=0 and res_ind01[i]=0
            arr_ind[i]=1 => res_ind01[i]=1 with probability arr_ratio_one and (res_ind01[i]+res_ind01[i]=1)
    """
    t01_ind01 = np.nonzero(arr_ind)[0]
    t01_ind02 = np.random.choice([0, 1], size=len(t01_ind01), p=[1 - arr_ratio_one, arr_ratio_one], replace=True)
    res_ind01 = np.zeros(arr_ind.shape, "bool")
    res_ind01[t01_ind01[t01_ind02 == 1]] = True
    res_ind00 = np.zeros(arr_ind.shape, "bool")
    res_ind00[t01_ind01[t01_ind02 == 0]] = True
    return res_ind01, res_ind00


#
def get_function_from_str(func_str):
    """
    Args:
        func_str: is like ghelper.split_bool_arr"
    """
    mod_name, func_name = func_str.rsplit('.', 1)
    mod = importlib.import_module(mod_name)
    func = getattr(mod, func_name)
    return func


# ################################################################################
# ### hdfs related
# ################################################################################
# def hdfs_list_and_match(
#         hdfs_cpoint,
#         hdfs_path,
#         regex_match_str,
#         regex_groupid,
#         get_full_path=False
# ):
#     """
#     Example: 
#         ghelper.hdfs_list_and_match(sysconf.conf["sys_hdfs_connection_point"],
#             "/tanlaDataByDate",
#             "sentDate=(\d+)",
#             1)
#     """
#     regex_match = re.compile(regex_match_str)
#     list01 = []
#     for s in hdfs.InsecureClient(hdfs_cpoint).list(hdfs_path):
#         s1 = regex_match.match(s)
#         if s1:
#             if get_full_path:
#                 list01.append(os.path.join(hdfs_path, str(s1.group(regex_groupid))))
#             else:
#                 list01.append(str(s1.group(regex_groupid)))

#     list01 = sorted(list01)
#     return list01


# #
# def hdfs_delete_filelist(hdfs_cpoint, filepath_list):
#     client = hdfs.InsecureClient(hdfs_cpoint)
#     for filepath in filepath_list:
#         client.delete(filepath, recursive=True)
#     return


################################################################################
###
################################################################################
def get_prev_datetime_str(s01_datetime, lag_from, lag_to, lag_step, lag_unit, input_format, output_format):
    """
    Get the previous lagged datetime strings of target_datetime

    Args:
        s01_datetime: is with format=input_format
        lag_from, lag_to: lag_from > lag_to
        lag_unit: use numpy.datetime64/timedelta64 format (M/d/m)
        input_format: use datetime.datetime.strptime format (e.g. %Y%m)
        output_format:

    Returns:
        dates02: previous lagged datetime strings
    """
    date01 = np.datetime64(datetime.datetime.strptime(s01_datetime, input_format), lag_unit)
    dates02 = []
    for i in xrange(lag_from, lag_to, lag_step):
        dates02.append(pd.to_datetime(str(date01 - np.timedelta64(i, lag_unit))).strftime(output_format))
    return dates02
#
def get_date_from_YW(input_YW, input_format="%YW%W", output_format="%Y%m%d"):
    """
    Example: get_date_from_YW('2016W10')
    """
    output = []
    for day_in_week in range(7):
        t0_dt = datetime.datetime.strptime(input_YW + '-%d' % day_in_week, "%s-%%w" % input_format)
        output.append(t0_dt.strftime(output_format))
    output = sorted(output)
    return output
#
def get_prev_YW_from_YW(input_YW, week_delta_list, input_format="%YW%W", output_format="%YW%W"):
    """
    Example: get_prev_YW_from_YW('2016W10', [-2,-1])
    """
    output = [((datetime.datetime.strptime(input_YW + '-0', "%s-%%w" % input_format) + datetime.timedelta(days=(i * 7)))
               .strftime(output_format))
              for i in week_delta_list]
    return output
#
def get_prev_YW_from_Ymd(input_Ymd, week_delta_list, input_format="%Y%m%d", output_format="%YW%W"):
    """
    Example: get_prev_YW_from_Ymd('20161001', [-2,-1])
    """
    output = [((datetime.datetime.strptime(input_Ymd, input_format) + datetime.timedelta(days=(i * 7)))
               .strftime(output_format))
              for i in week_delta_list]
    return output
#
def get_prev_Ymd_from_YW(input_YW, week_delta_list, input_format="%YW%W", output_format="%Y%m%d"):
    """
    Example: get_prev_Ymd_from_YW('2016W10', [-2,-1])
    """
    output01 = [get_date_from_YW(prev_YW, input_format, output_format)
                for prev_YW in get_prev_YW_from_YW(input_YW, week_delta_list, input_format, input_format)]
    output02 = [x01 for l01 in output01 for x01 in l01]
    return output02
#
def get_last_day_of_month_from_Ym(input_Ym, delta_month=0, input_format="%Y%m", output_format="%Y%m%d"):
    """
    Example: get_last_day_of_month_from_Ym('20161001', -1)
    """
    date01_nextmonth = (datetime.datetime.strptime(str(input_Ym) + '-28', input_format + "-%d")
                        + dateutil.relativedelta.relativedelta(months=delta_month)
                        + datetime.timedelta(days=4))
    date02_lastdayofmonth = date01_nextmonth - datetime.timedelta(days=date01_nextmonth.day)
    return date02_lastdayofmonth.strftime(output_format)
#
def get_last_week_of_month_from_Ym(input_Ym, delta_month=0, input_format="%Y%m", output_format="%YW%W"):
    """
    Example: get_last_week_of_month_from_Ym('201610', -1)
    """
    date01 = (datetime.datetime.strptime(str(input_Ym), input_format)
              + dateutil.relativedelta.relativedelta(months=delta_month + 1)
              - dateutil.relativedelta.relativedelta(days=7))
    return date01.strftime(output_format)
#
def get_prev_nmonths(upto_month, nmonth_lookback=2):
    """
    Example: get_prev_nmonths('201610', 2)
    """
    ym = []
    for i in range(-nmonth_lookback, 0, 1):
        ym.append(get_last_day_of_month_from_Ym(upto_month, delta_month=i + 1)[0:6])
    return ym
#
def get_prev_nmonths_upto_ymd(upto_ymd, nmonth_lookback=6):
    """
    Example: get_prev_nmonths('201610', 2)
    """
    upto_month = upto_ymd[0:6]
    last_day = get_last_day_of_month_from_Ym(upto_month)
    if last_day == upto_ymd:
        delta = 0
    else:
        delta = -1
    ym = []
    for i in range(-nmonth_lookback+delta, 0+delta, 1):
        ym.append(get_last_day_of_month_from_Ym(upto_month, delta_month=i+1)[0:6])
    return ym
#
def get_prev_YW_upto_Ymd(input_Ymd, week_delta_list, input_format="%Y%m%d", output_format="%YW%W"):
    """
    Example: get_prev_YW_from_Ymd('20161001', [-2,-1])
    """
    # next_input_Ymd = datetime.datetime.strptime(input_Ymd, input_format) + datetime.timedelta
    # input_Yms = ['201512', '201601', '201602', '201603']
    # input_Ymd_list = [ghelper.get_last_day_of_month_from_Ym(input_Ym) for input_Ym in input_Yms]
    # date_input_Ymd_list = [datetime.datetime.strptime(input_Ymd, '%Y%m%d') for input_Ymd in input_Ymd_list]
    # input_Ymd_list = [datetime.datetime.strftime(date_input_Ymd + datetime.timedelta(1), '%Y%m%d')
    #                    for date_input_Ymd in date_input_Ymd_list]
    
    next_input_Ymd = (datetime.datetime.strptime(input_Ymd, input_format) + datetime.timedelta(days=1)).strftime(input_format)
    output = [((datetime.datetime.strptime(next_input_Ymd, input_format) + datetime.timedelta(days=(i * 7)))
               .strftime(output_format)) for i in week_delta_list]
    return output
#
def get_wdate_from_lastwdate(str_ymd):
    yw = datetime.datetime.strptime(str_ymd, '%Y%m%d').strftime('%YW%W')
    dates = get_date_from_YW(yw)
    return dates
#
def get_prev_ndays(upto_ymd, ndays_look_back):
    date = datetime.datetime.strptime(upto_ymd, '%Y%m%d')
    dates = []
    curr_date = date
    for i in range(ndays_look_back):
        dates.append(curr_date.strftime('%Y%m%d'))
        curr_date -= datetime.timedelta(1)
    dates = sorted(dates)
    return dates
#
def get_lastwdate_one_year(upto_ymd):
    """
    Get the last dates of each week in one year, mostly for leadgen features
    :param upto_ymd: 
    :return: 
    """
    dates = []
    end_date = datetime.datetime.strptime(upto_ymd, '%Y%m%d') - datetime.timedelta(days=360)
    curr_date = datetime.datetime.strptime(upto_ymd)

    while curr_date >= end_date:
        dates.append(curr_date.strftime('%Y%m%d'))
        curr_date = curr_date - datetime.timedelta(days=7)
    return dates
#
def get_all_days_of_month(input_Ym):
    """
    Get all dates in a month
    """
    y = int(input_Ym[0:4])
    m = int(input_Ym[4:6])
    last_date = calendar.monthrange(y, m)[1]
    dates = [datetime.date(y, m, d).strftime('%Y%m%d') for d in range(1, last_date + 1)]
    return dates
#
def get_last_dow_upto_Ymd(input_Ymd, delta_week):
    """
    get_last_dow_upto_Ymd("20180506", 0) # '20180506'
    get_last_dow_upto_Ymd("20180506", -1) # '20180429'
    get_last_dow_upto_Ymd("20180503", 0) # '20180429'
    """
    date01=datetime.datetime.strptime(input_Ymd, "%Y%m%d")
    date02=date01+datetime.timedelta(days=-date01.weekday() -1 + (date01.weekday()/6+delta_week)*7)
    return date02.strftime("%Y%m%d")

################################################################################
### Some statistical helper
################################################################################
def get_gain_table(pdf01,
                   gt_cname_target,
                   gt_cname_target_val,
                   gt_num_bins,
                   gt_natural_rate,
                   method="v02"):
    """
    Get the gain table
    Args:
        pdf01: dataframe which must have column 
            "pred_proba": prediction probability for the positve case
            "score02" (optional): one-to-one mapping with "pred_proba"
            a label column named by variable gt_cname_target
                Label of interest would corresponds with (pdf01[gt_cname_target]=gt_cname_target_val)
            NOTE THAT THE POSITIVE CLASS (IMPLIED BY "pred_proba" and "score02") 
                MAY NOT THE SAME AS (pdf01[gt_cname_target]=gt_cname_target_val)
        Assume pdf01 is already sorted by descending (or ascending) score of positive class
    Example: 
        get_gain_table(
            pdf01=pdf01,
            gt_cname_target="label", 
            gt_cname_target_val=1, 
            gt_num_bins=10, 
            gt_natural_rate=None,
            method="v02")
    """

    #
    def target_count(x01, gt_cname_target_val=gt_cname_target_val):
        return (x01 == gt_cname_target_val).sum()

    if method == "v01":
        #
        pdf01["rank"] = (np.floor_divide(np.arange(pdf01.shape[0]),
                                         math.ceil(float(pdf01.shape[0]) / gt_num_bins)).astype("i4"))
        if gt_natural_rate is None:
            gt_natural_rate = float((pdf01[gt_cname_target] == gt_cname_target_val).sum()) / pdf01.shape[0]
        #
        pdf02 = pdf01.groupby("rank", as_index=False).agg({gt_cname_target: ["count", target_count]})
        pdf02.columns = ["%s" % (s02) if len(s02) > 0 else s01
                         for s01, s02 in zip(pdf02.columns.get_level_values(0), pdf02.columns.get_level_values(1))]
        pdf02["ccount"] = np.cumsum(pdf02["count"])
        pdf02["target_ccount"] = np.cumsum(pdf02["target_count"])
        pdf02["pos_rate"] = pdf02["target_count"].astype("f4") / pdf02["count"]
        pdf02["pos_crate"] = pdf02["target_ccount"].astype("f4") / pdf02["ccount"]
        pdf02["pos_gain"] = pdf02["pos_rate"] / gt_natural_rate
        pdf02["pos_cgain"] = pdf02["pos_crate"] / gt_natural_rate
    elif method == "v02":
        #
        pdf01["rank"] = (np.floor_divide(np.arange(pdf01.shape[0]),
                                         math.ceil(float(pdf01.shape[0]) / gt_num_bins)).astype("i4"))
        #
        if "score" in pdf01.columns:
            pdf02 = (pdf01.groupby("rank", as_index=False)
                .agg({
                gt_cname_target: ["count", target_count],
                "pred_proba": ["min", "max"],
                "score": ["min", "max"]
            })
            )
        else:
            pdf02 = (pdf01.groupby("rank", as_index=False)
                .agg({
                gt_cname_target: ["count", target_count],
                "pred_proba": ["min", "max"],
            })
            )
        pdf02.columns = [
            "%s_%s" % (s01, s02) if (s01 == "score") or (s01 == "pred_proba")
            else "%s" % (s02) if len(s02) > 0 else s01
            for s01, s02 in zip(pdf02.columns.get_level_values(0), pdf02.columns.get_level_values(1))]
        pdf02["target_count_prow"] = pdf02["target_count"].values.astype("f4") / pdf02["count"]
        pdf02["target_count_pcol"] = pdf02["target_count"].values.astype("f4") / pdf02["target_count"].sum()
        #
        pdf02["ccount"] = np.cumsum(pdf02["count"])
        pdf02["target_ccount"] = np.cumsum(pdf02["target_count"])
        pdf02["target_ccount_prow"] = pdf02["target_ccount"].values.astype("f4") / pdf02["ccount"]
        pdf02["target_ccount_pcol"] = pdf02["target_ccount"].values.astype("f4") / pdf02["target_count"].sum()
        #
        pdf02["pos_gain"] = (pdf02["target_count_pcol"]
                             / (pdf02["count"].values.astype("f4") / pdf02["count"].sum()))
        pdf02["pos_cgain"] = (pdf02["target_ccount_pcol"]
                              / (pdf02["count"].values.cumsum().astype("f4") / pdf02["count"].sum()))
    else:
        pdf02 = None
    #
    return pdf02

In [22]:
pd00[['date']].head(2)

date
0  2016-08-13
1  2016-10-11

In [24]:
import datetime
def get_week(date):
    y,m, d = date.split("-")
    return datetime.date(int(y), int(m), int(d)).isocalendar()[1]


pd00['week'] = pd00['date'].apply(get_week)
pd00[['week','date']].head(4)

week        date
0    32  2016-08-13
1    41  2016-10-11
2    45  2016-11-08
3    49  2016-12-10